In [ ]:
from sedona.spark import SedonaContext
from pyspark.sql.functions import expr

# Start Spark + Sedona
config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    config("spark.sql.legacy.parquet.nanosAsLong", "true"). \
    getOrCreate()
sedona = SedonaContext.create(config)


25/05/25 22:25:52 WARN Utils: Your hostname, cs-spatial-501 resolves to a loopback address: 127.0.0.1; using 128.101.33.153 instead (on interface eno1)
25/05/25 22:25:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/huan1531/.ivy2/cache
The jars for the packages stored in: /home/huan1531/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
edu.ucar#cdm-core added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-90f7009d-f68c-429f-8d86-193945b63205;1.0
	confs: [default]


:: loading settings :: url = jar:file:/export/scratch/huan1531/sedona_experiments/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

25/05/25 22:26:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [12]:
sdf = sedona.read.parquet(f"/export/scratch/huan1531/sedona_experiments/data/gl_*.parquet")

In [13]:
# Create a geometry column for spatial queries
sdf = sdf.withColumn("geom", expr("ST_Point(longitude, latitude)")).withColumn("timestamp", expr("to_timestamp(time/1000000000 )"))
sdf.createOrReplaceTempView("GL")
sdf.show()

+---------+--------+-------------------+------------------+--------------+-------------------+
|longitude|latitude|               time|               t2m|          geom|          timestamp|
+---------+--------+-------------------+------------------+--------------+-------------------+
|    -75.0|    84.0|1577836800000000000| 242.0457332342676|POINT (-75 84)|2019-12-31 18:00:00|
|    -75.0|    84.0|1577840400000000000|242.03962641346033|POINT (-75 84)|2019-12-31 19:00:00|
|    -75.0|    84.0|1577844000000000000| 241.9928074539378|POINT (-75 84)|2019-12-31 20:00:00|
|    -75.0|    84.0|1577847600000000000| 241.8788134655351|POINT (-75 84)|2019-12-31 21:00:00|
|    -75.0|    84.0|1577851200000000000|241.77906872568278|POINT (-75 84)|2019-12-31 22:00:00|
|    -75.0|    84.0|1577854800000000000| 241.6345406332437|POINT (-75 84)|2019-12-31 23:00:00|
|    -75.0|    84.0|1577858400000000000| 241.5307246795198|POINT (-75 84)|2020-01-01 00:00:00|
|    -75.0|    84.0|1577862000000000000|241.406552

In [27]:
# get variable: daily-0.25

max_lat = 80
min_lat = 60
min_lon = -60
max_lon = -30
start_time = "2020-02-01"
end_time = "2020-04-30"

query = f"""
    SELECT to_date(timestamp), latitude, longitude, avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY to_date(timestamp), latitude, longitude
    """
result = sedona.sql(query)
result.show()

+------------------+--------+---------+------------------+
|to_date(timestamp)|latitude|longitude|          avg(t2m)|
+------------------+--------+---------+------------------+
|        2020-02-24|    80.0|    -60.0|234.16912183024797|
|        2020-03-17|    80.0|    -60.0|240.86584456412655|
|        2020-02-29|   78.75|    -60.0|228.37222217893188|
|        2020-03-31|   78.75|    -60.0| 244.0321463357487|
|        2020-04-03|    78.5|    -60.0|248.27180668120687|
|        2020-04-25|    78.5|    -60.0|262.82037426805334|
|        2020-04-02|   78.25|    -60.0| 247.9460247545293|
|        2020-04-23|   78.25|    -60.0|260.14626529010735|
|        2020-04-22|   77.25|    -60.0| 259.8801096832564|
|        2020-03-07|    77.0|    -60.0| 236.4869147774799|
|        2020-02-01|   76.25|    -60.0|245.71211577644183|
|        2020-02-23|    75.5|    -60.0|244.54952976525485|
|        2020-03-06|   73.75|    -60.0|  241.070338244215|
|        2020-04-23|   73.75|    -60.0| 266.093206135980

In [28]:
# get variable: monthly-0.25
query = f"""
    SELECT month(to_date(timestamp)), latitude, longitude, avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY month(to_date(timestamp)), latitude, longitude
    """
result = sedona.sql(query)
result.show()

+-------------------------+--------+---------+------------------+
|month(to_date(timestamp))|latitude|longitude|          avg(t2m)|
+-------------------------+--------+---------+------------------+
|                        3|   77.75|    -60.0| 235.5000870782407|
|                        4|   77.75|   -59.75|252.25854846257278|
|                        2|    61.0|    -59.5| 263.4791680356976|
|                        4|    76.5|   -59.25|  258.260846548734|
|                        4|   60.25|   -59.25| 272.4420701969899|
|                        4|   79.25|    -59.0|254.85805064533147|
|                        2|   63.25|    -60.0|257.16650817592296|
|                        2|   73.25|   -59.75|245.00487690350477|
|                        4|   65.75|   -59.75|264.64021380487463|
|                        3|   63.25|   -59.75| 260.8396948156056|
|                        4|   79.75|    -59.5| 256.0343978120856|
|                        3|    64.0|   -59.25|258.08010221844836|
|         

In [29]:
# yearly-0.25
query = f"""
    SELECT year(to_date(timestamp)), latitude, longitude, avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY year(to_date(timestamp)), latitude, longitude
    """
result = sedona.sql(query)
result.show()

+------------------------+--------+---------+------------------+
|year(to_date(timestamp))|latitude|longitude|          avg(t2m)|
+------------------------+--------+---------+------------------+
|                    2020|   78.75|    -59.5| 242.0474391033751|
|                    2020|   78.25|    -59.5|241.14905695605995|
|                    2020|   63.75|    -59.0| 262.2782557364784|
|                    2020|   75.75|   -59.25|255.59577939215256|
|                    2020|   65.75|    -59.5|256.67219804738846|
|                    2020|    73.0|   -59.25|252.37323254293588|
|                    2020|    61.0|   -59.25| 267.6983546327912|
|                    2020|   69.75|    -60.0|252.76973179764155|
|                    2020|    66.5|    -60.0|255.26661336424402|
|                    2020|   77.25|    -59.0|242.02891946405828|
|                    2020|   65.75|    -60.0|256.25186618114276|
|                    2020|   74.25|    -59.5|251.30813266278165|
|                    2020

In [33]:
# daily-0.5
query = f"""
    SELECT to_date(timestamp), int(latitude/0.5)*0.5, int(longitude/0.5)*0.5, avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY to_date(timestamp), int(latitude/0.5), int(longitude/0.5)
    """
result = sedona.sql(query)
result.show()

+------------------+------------------------+-------------------------+------------------+
|to_date(timestamp)|((latitude / 0.5) * 0.5)|((longitude / 0.5) * 0.5)|          avg(t2m)|
+------------------+------------------------+-------------------------+------------------+
|        2020-02-11|                    78.5|                    -60.0| 238.4643372816614|
|        2020-02-14|                    78.0|                    -60.0| 240.1958754313939|
|        2020-04-26|                    75.5|                    -60.0| 271.9186466928749|
|        2020-03-13|                    74.0|                    -60.0|240.63471835996197|
|        2020-03-12|                    73.5|                    -60.0| 242.3185893806153|
|        2020-03-18|                    70.0|                    -60.0|260.31975837290315|
|        2020-04-01|                    69.5|                    -60.0|254.16102958875535|
|        2020-03-27|                    68.0|                    -60.0|262.56041029896545|

In [34]:
# daily-1
query = f"""
    SELECT to_date(timestamp), int(latitude), int(longitude), avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY to_date(timestamp), int(latitude), int(longitude)
    """
result = sedona.sql(query)
result.show()

+------------------+--------+---------+------------------+
|to_date(timestamp)|latitude|longitude|          avg(t2m)|
+------------------+--------+---------+------------------+
|        2020-03-15|      79|      -60|239.94068241606172|
|        2020-04-28|      77|      -60|253.16930733473893|
|        2020-02-27|      76|      -60|242.33669780064804|
|        2020-03-07|      74|      -60|241.28026020946547|
|        2020-02-09|      71|      -60| 248.0965324423421|
|        2020-02-03|      70|      -60|243.20161870595766|
|        2020-04-04|      70|      -60| 258.2664458976479|
|        2020-03-26|      66|      -60| 258.4145363022247|
|        2020-02-28|      65|      -60|246.90343353135765|
|        2020-04-27|      63|      -60| 270.3412421558096|
|        2020-02-12|      78|      -59|238.10305945904133|
|        2020-03-04|      76|      -59|245.64609107752256|
|        2020-04-18|      72|      -59| 264.8458243733756|
|        2020-02-17|      68|      -59| 251.549420105008

In [35]:
# heatmap: 1-degree
query = f"""
    SELECT int(latitude), int(longitude), avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY int(latitude), int(longitude)
    """
result = sedona.sql(query)
result.show()


+--------+---------+------------------+
|latitude|longitude|          avg(t2m)|
+--------+---------+------------------+
|      78|      -60|241.55426257596383|
|      79|      -59|243.39979221247552|
|      80|      -60|245.02903432356433|
|      65|      -60| 256.6135375925568|
|      67|      -59|254.57154573881886|
|      60|      -59|267.78185513876014|
|      71|      -59| 252.5342646807456|
|      61|      -59| 267.2709019542673|
|      68|      -60| 253.4579179923501|
|      66|      -59|255.93185669157108|
|      69|      -60|252.97881679108085|
|      64|      -59|259.37367597977527|
|      65|      -59| 257.3407471513011|
|      62|      -59|265.92550431203324|
|      80|      -59|245.28258103225724|
|      69|      -59|253.42958728272345|
|      76|      -60|251.13748881024665|
|      72|      -60|251.85966659118375|
|      70|      -59| 252.9453369655827|
|      63|      -59|262.85068155820807|
+--------+---------+------------------+
only showing top 20 rows



In [37]:
# find time: daily-0.25
query = f"""
    SELECT to_date(timestamp), avg(t2m)
    FROM GL
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
        AND longitude BETWEEN {min_lon} AND {max_lon}
        AND timestamp BETWEEN '{start_time}' AND '{end_time}'
    GROUP BY to_date(timestamp)
    HAVING avg(t2m) > 235
    """
result = sedona.sql(query)
result.show()

+------------------+------------------+
|to_date(timestamp)|          avg(t2m)|
+------------------+------------------+
|        2020-04-30|258.84280067809146|
|        2020-03-07|247.73517112681435|
|        2020-03-13|244.07699351090926|
|        2020-02-04|249.53503806179583|
|        2020-02-15|251.39328894421308|
|        2020-02-12|247.83334599577054|
|        2020-04-29| 259.5269189509637|
|        2020-04-26| 264.4540002987955|
|        2020-04-21| 264.1552609010633|
|        2020-03-09|248.81328597521448|
|        2020-03-10| 246.8260945325636|
|        2020-03-11| 245.6889452570519|
|        2020-04-19| 258.9424783155384|
|        2020-02-11|248.75475794885722|
|        2020-04-13| 261.7828117812596|
|        2020-02-25| 242.1614716602057|
|        2020-03-29| 260.2015067611161|
|        2020-02-09|252.27650145548768|
|        2020-03-21|249.89361785108426|
|        2020-04-14|259.69713384241265|
+------------------+------------------+
only showing top 20 rows



In [ ]:
sedona.stop()